In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

import polars as pl
import japanize_matplotlib

data_dir = "/home/ohta/dev/bayesian_statistics/data"

obsidian_path = "11_gdf_obsidian.csv"
sites_path = "12_gdf_sites_with_ratio.csv"
elevation_path = "12_gdf_elevation_with_ratio.csv"

df_obsidian = pl.read_csv(os.path.join(data_dir, obsidian_path))
df_sites = pl.read_csv(os.path.join(data_dir, sites_path))
df_elevation = pl.read_csv(os.path.join(data_dir, elevation_path))

time_period_name = {
    0: "早期・早々期",
    1: "前期",
    2: "中期",
    3: "後期",
    4: "晩期"
}

origin_order = ["神津島", "信州", "箱根", "高原山", "その他"]

def plot_contour(df, x_col='x', y_col='y', value_col='cost_kouzu', figsize=(12, 8), plot_probability=False, n_levels=30):

    # まず海陸判定のマスクを作成
    mask = (
        df
        .with_columns(
            (~pl.col('is_sea')).alias("is_not_sea")
        )
        .pivot(
            values='is_not_sea',
            on=x_col,
            index=y_col
        )
    )

    # 値のピボットテーブルを作成
    grid_data = (
        df
        .pivot(
            values=value_col,
            on=x_col,
            index=y_col
        )
    )

    # マスクを適用（y_col列は保持）
    grid_data = grid_data.with_columns(
        [
            pl.col(col) * mask.get_column(col)
            for col in grid_data.columns
            if col != y_col
        ]
    )
    
    # メッシュグリッドの作成
    x_mesh = np.array(grid_data.columns[1:], dtype=float)
    y_mesh = np.array(grid_data.to_numpy()[:, 0], dtype=float)
    values_mesh = grid_data.to_numpy()[:, 1:]
    
    # プロット作成
    fig, ax = plt.subplots(figsize=figsize)
    
    if plot_probability:
        # 確率表示モード（0-1の範囲）
        contour = ax.contourf(
            x_mesh, y_mesh, values_mesh,
            levels=np.linspace(0, 1, n_levels + 1),
            cmap='Blues',
            alpha=0.7,
            vmin=0,
            vmax=1
        )
    else:
        # 通常モード（データをそのまま使用）
        contour = ax.contourf(
            x_mesh, y_mesh, values_mesh, 
            levels=n_levels, 
            cmap='Blues',
            alpha=0.7
        )
    
    # カラーバーの追加
    if plot_probability:
        # カラーバー（0-1の範囲に固定）
        plt.colorbar(contour, ax=ax, label='Ratio', ticks=np.linspace(0, 1, 6))
    else:
        plt.colorbar(contour, ax=ax)
    
    # ラベルの設定
    ax.set_xlabel('経度')
    ax.set_ylabel('緯度')
    
    return fig, ax

# 通常プロット（データそのまま）

period = 0
origin = "神津島"

for period in range(5):
    for origin in origin_order[:-1]:


        fig, ax = plot_contour(df_elevation, value_col=f"ratio_{period}_{origin}", plot_probability=True, n_levels=40)

        boundary_df = df_elevation.filter(pl.col("is_sea") == 0, pl.col("average_elevation").is_null())

        ax.scatter(
            boundary_df["x"], 
            boundary_df["y"], 
            c="black", 
            s=0.001
        )

        ax.scatter(
            df_sites["経度"], 
            df_sites["緯度"], 
            c=df_sites[f"比率_{period}_{origin}"], 
            cmap="Blues", 
            edgecolors="black", 
            linewidths=0.5,
            vmin=0,
            vmax=1 
        )

        ax.set_title(f"黒曜石の産地構成比 ({time_period_name[period]}, {origin})")

        fig.savefig(f"../output/13_visualize_obsidian_first_model/obsidian_ratio_{period}_{origin}.png")
        plt.close()

In [2]:
# 全時期・産地のプロットを1つの図にまとめる
fig, axes = plt.subplots(4, 5, figsize=(30, 20))

# 産地ごとにプロット
for origin_idx, origin in enumerate(origin_order[:-1]):  # その他を除く
    for period in range(5):
        ax = axes[origin_idx, period]
        
        # コンターの作成
        grid_data = (
            df_elevation
            .pivot(
                values=f"ratio_{period}_{origin}",
                on='x',
                index='y'
            )
        )
        
        # マスクの適用
        mask = (
            df_elevation
            .with_columns(
                (~pl.col('is_sea')).alias("is_not_sea")
            )
            .pivot(
                values='is_not_sea',
                on='x',
                index='y'
            )
        )
        
        grid_data = grid_data.with_columns(
            [
                pl.col(col) * mask.get_column(col)
                for col in grid_data.columns
                if col != 'y'
            ]
        )
        
        # メッシュグリッドの作成
        x_mesh = np.array(grid_data.columns[1:], dtype=float)
        y_mesh = np.array(grid_data.to_numpy()[:, 0], dtype=float)
        values_mesh = grid_data.to_numpy()[:, 1:]
        
        # コンタープロット
        contour = ax.contourf(
            x_mesh, y_mesh, values_mesh,
            levels=np.linspace(0, 1, 41),
            cmap='Blues',
            alpha=0.7,
            vmin=0,
            vmax=1
        )
        
        # 境界線のプロット
        boundary_df = df_elevation.filter(pl.col("is_sea") == 0, pl.col("average_elevation").is_null())
        ax.scatter(
            boundary_df["x"],
            boundary_df["y"],
            c="black",
            s=0.001
        )
        
        # 遺跡のプロット
        scatter = ax.scatter(
            df_sites["経度"],
            df_sites["緯度"],
            c=df_sites[f"比率_{period}_{origin}"],
            cmap="Blues",
            edgecolors="black",
            linewidths=0.5,
            vmin=0,
            vmax=1
        )
        
        # フレームと目盛りの削除
        ax.set_frame_on(False)
        ax.set_xticks([])
        ax.set_yticks([])

# レイアウトの調整
plt.tight_layout()
# 保存
fig.savefig("../output/13_visualize_obsidian_first_model/obsidian_ratio_all.png", dpi=300, bbox_inches='tight')
plt.close()